In [18]:
%pip install langchain
%pip install langgraph
%pip install ag-ui-protocol

/Users/Guruprasad.BR/workspace/RD/fds-documentation-explorer/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/Users/Guruprasad.BR/workspace/RD/fds-documentation-explorer/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/Users/Guruprasad.BR/workspace/RD/fds-documentation-explorer/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
import pickle,os, traceback,json

all_events=[]
with open("all_events.pkl", "rb") as file:
    all_events=pickle.load(file)
# 
print(json.dumps(all_events[-2],default=str,indent=2))
print(len(all_events))



{
  "event": "on_chain_stream",
  "run_id": "ac1d7222-c082-46e4-b48d-313b85989f32",
  "name": "fds_agent",
  "tags": [],
  "metadata": {
    "thread_id": "87f75271-02b7-4f57-a3dd-c79160b4a577",
    "user_id": "guru"
  },
  "data": {
    "chunk": {
      "end_conv": {
        "messages": [
          "content='implement a page to showcase simple button component from Fabric Design System' additional_kwargs={} response_metadata={} id='9da8fca0-3073-4d58-b46c-f2b4b4760bfb'",
          "content=[{'type': 'text', 'text': \"I'll help you implement a page to showcase a simple button component from the Fabric Design System. Let me start by executing the plan executor agent to handle this task.\", 'index': 0}, {'type': 'tool_use', 'name': 'plan_executor_agent', 'id': 'tooluse_DN_L0fuwQfSTbNyt6ucIQQ', 'index': 1, 'input': ''}] additional_kwargs={} response_metadata={'stopReason': 'tool_use', 'metrics': {'latencyMs': 1591}, 'model_name': 'us.anthropic.claude-sonnet-4-20250514-v1:0', 'ResponseMetad

In [21]:
%xmode Verbose
import nest_asyncio
nest_asyncio.apply()
import copy

from typing import Dict,List,TypedDict,Any,Optional
from enum import Enum
import pickle,os, traceback,json
from langgraph.types import Command,Interrupt
from langchain_core import messages as langchain_messages
from datetime import datetime, timezone
from ag_ui.core import EventType
from ag_ui.encoder import EventEncoder

from ag_ui.core import (
    EventType,
    CustomEvent,
    MessagesSnapshotEvent,
    RawEvent,
    RunAgentInput,
    RunErrorEvent,
    RunFinishedEvent,
    RunStartedEvent,
    StateDeltaEvent,
    StateSnapshotEvent,
    StepFinishedEvent,
    StepStartedEvent,
    TextMessageContentEvent,
    TextMessageEndEvent,
    TextMessageStartEvent,
    ToolCallArgsEvent,
    ToolCallEndEvent,
    ToolCallStartEvent,
    ThinkingTextMessageStartEvent,
    ThinkingTextMessageContentEvent,
    ThinkingTextMessageEndEvent,
    ThinkingStartEvent,
    ThinkingEndEvent,
)



# event_types {'on_chain_start': 9, 'on_chain_stream': 16, 'on_chain_end': 9, 'on_chat_model_start': 3, 'on_chat_model_stream': 566, 'on_chat_model_end': 3, 'on_tool_start': 2, 'on_tool_end': 2}


class EventTrackType(str, Enum):
    REGISTERED = "registered"
    START = "start"
    END = "end"


class TrackEvent(TypedDict):
    state: EventTrackType
    type: EventType


class LangGraphToAgUi:

    def __init__(self):
        self.ref={"wrap_status":None,"last_type":None,"last_id":None}
        self.tool_use_id=None

        self.event_types={}
        self.chunk_types={}
        self.parsed_events={}
        self.unparsed_events={}
        self.filtered_event=None
        self.track_event:Dict[str,TrackEvent]={} # mostly all events are sequential except tool calls
        
        # Buffer for tool events per run_id
        self.tool_event_buffer: Dict[str, List[Any]] = {}
        self.active_tool_runs: Dict[str, bool] = {}

    def __get_filtered_data(self,event):
        if "event" in event:
            self.track_event[event["run_id"]]=self.track_event.get(event["run_id"],{})
            _type=event["event"]
            self.event_types[_type]=self.event_types.get(_type,0)+1
            _contents=None
            if isinstance(event.get("data",{}).get("chunk",{}),langchain_messages.BaseMessage):
                ai_msg:langchain_messages.AIMessage=event.get("data",{}).get("chunk",{})
                _contents=ai_msg.content
                self.unparsed_events[_type]=self.unparsed_events.get(_type,0)+1            
            elif event.get("data") and event["data"].get("output") and type(event["data"]["output"])==Command:  
                self.unparsed_events[_type]=self.unparsed_events.get(_type,0)+1           
                return None
            elif (event.get("data") and event["data"].get("chunk") and type(event["data"]["chunk"])==Command):
                self.unparsed_events[_type]=self.unparsed_events.get(_type,0)+1   
                return None
            elif event.get("data",{}).get("chunk",{}).get("__interrupt__",False):
                _interrupt:Interrupt= event["data"]["chunk"]["__interrupt__"]
                _contents={
                    "type": "interrupt",
                    "value":_interrupt[0].value,
                    "id": _interrupt[0].id,
                }            
            elif _type in ["on_tool_start","on_tool_end"]:
                pass
            elif not isinstance(event.get("data",{}).get("chunk",{}),langchain_messages.BaseMessage):
                if event.get("data",{}).get("chunk",{}).get("start_conv",True):
                    self.unparsed_events[_type]=self.unparsed_events.get(_type,0)+1   
                    return None
                if event.get("data",{}).get("chunk",{}).get("llm",True):
                    self.unparsed_events[_type]=self.unparsed_events.get(_type,0)+1   
                    return None
                if event.get("data",{}).get("chunk",{}).get("route",True):
                    self.unparsed_events[_type]=self.unparsed_events.get(_type,0)+1   
                    return None      
                print("Unhandled", event)    
            elif event.get("data") and event["data"].get("input") and event["data"]["input"].get("store"):
                event["data"]["input"]["store"] = "Accessing to store information"   
            else:
                _contents=event.get("data",{}).get("chunk",{}).get("content")
            self.parsed_events[_type]=self.parsed_events.get(_type,0)+1
            if _type=="on_chat_model_stream" and _contents is not None and type(_contents) is list:
                for _content in _contents:
                    if "type" in _content:
                        self.chunk_types[_type]=self.chunk_types.get(_type,set()) | {_content["type"]}
            else:
                self.chunk_types[_type]=self.chunk_types.get(_type,0)+1

        return event

    
    def __process_chunk(self,event,_type,_content):
        last_type=self.ref.get("last_type",_type)
        last_id:str=self.ref.get("last_id",event["run_id"]) or event["run_id"]
        if not last_id.endswith(event["run_id"]):
            print(event["run_id"],last_id)
        ac_events=[]
        if ((_type!= last_type and last_type is not None) or not last_id.endswith(event["run_id"])) and self.ref["wrap_status"]=="started":
            if last_type=="text":
                ac_events.append(TextMessageEndEvent(
                    type=EventType.TEXT_MESSAGE_END,
                    message_id=self.ref["last_id"] or ""
                ))
            elif last_type=="reasoning_content":
                ac_events.append(CustomEvent(
                    type=EventType.CUSTOM,
                    name=last_type,
                    value={"text":"","type":EventType.THINKING_TEXT_MESSAGE_END,"message_id":self.ref["last_id"] or ""}
                ))
            elif last_type=="tool_use":
                ac_events.append(CustomEvent(
                    type=EventType.CUSTOM,
                    name=last_type,
                    value={"text":"","type":EventType.TEXT_MESSAGE_END,"message_id":self.ref["last_id"] or ""}
                ))
            self.ref["wrap_status"]="ended"
            self.ref["last_id"]=None
        if ((_type!= last_type) or not last_id.endswith(event["run_id"])) and self.ref["wrap_status"] != "started":
            if _type=="text":
                self.ref["last_id"]="text_"+event["run_id"]
                ac_events.append(TextMessageStartEvent(
                    type=EventType.TEXT_MESSAGE_START,
                    role='assistant',
                    message_id=self.ref["last_id"] or ""
                ))
            elif _type=="reasoning_content":
                self.ref["last_id"]="reasoning_content_"+event["run_id"]
                ac_events.append(CustomEvent(
                    type=EventType.CUSTOM,
                    name=_type,
                    value={"text":"","type":EventType.THINKING_TEXT_MESSAGE_START,"message_id":self.ref["last_id"] or ""},
                ))
            elif _type=="tool_use":
                self.ref["last_id"]="tool_use_"+event["run_id"]
                ac_events.append(CustomEvent(
                    type=EventType.CUSTOM,
                    name=_type,
                    value={"text":"","type":EventType.TEXT_MESSAGE_START,"message_id":self.ref["last_id"] or ""},
                ))
            self.ref["wrap_status"]="started"    
        if _content:
            if _type=="text":
                if _content.get("text"):
                    ac_events.append(TextMessageContentEvent(
                        type=EventType.TEXT_MESSAGE_CONTENT,
                        message_id="text_"+event["run_id"],
                        # raw_event=event,
                        delta= _content.get("text"),
                    ))
            elif _type=="tool_use":
                if _content.get("id") and _content.get("name"):
                    ac_events.append(CustomEvent(
                        type=EventType.CUSTOM,
                        # raw_event=event,
                        name=_type,
                        value= {"text":f"Proposed Tool Call: Name: {_content["name"]}, Id: {_content["id"]}","type":EventType.TEXT_MESSAGE_CONTENT,"message_id":"tool_use_"+event["run_id"]}
                    ))
                elif _content.get("input"):
                    ac_events.append(CustomEvent(
                        type=EventType.CUSTOM,
                        # raw_event=event,
                        name=_type,
                        value= {"text":f"Arguments: {_content["input"]}","type":EventType.TEXT_MESSAGE_CONTENT,"message_id":"tool_use_"+event["run_id"]}
                    ))   
            elif _type=="reasoning_content":
                reasoning_text=_content.get("reasoning_content",{}).get("text")
                if reasoning_text:
                    ac_events.append(CustomEvent(
                        type=EventType.CUSTOM,
                        name=_type,
                        value= {"text":reasoning_text,"type":EventType.THINKING_TEXT_MESSAGE_CONTENT,"message_id":"reasoning_content_"+event["run_id"]}
                    ))   
            elif _type=="code" or _type=="plan":
                ac_events.append(CustomEvent(
                    type=EventType.CUSTOM,
                    name=_type,
                    value= {"type":_type+"_start","message_id":_type+"_"+event["run_id"]}
                ))   
                ac_events.append(CustomEvent(
                    type=EventType.CUSTOM,
                    name=_type,
                    value= {"text":_content.get("text",{}),"type":_type,"message_id":_type+"_"+event["run_id"]}
                ))   
                ac_events.append(CustomEvent(
                    type=EventType.CUSTOM,
                    name=_type,
                    value= {"type":_type+"_end","message_id":_type+"_"+event["run_id"]}
                ))
            else:
                print("Unhandled chunk type:", _type, _content)
        return ac_events
    
    def __process_chunks(self,event):
        chunks=[]
        if event["event"]=="on_chat_model_stream" or event["event"]=="on_custom_event": # if its still chunk, close other type if any before creating new chunk
            base_message:langchain_messages.BaseMessage=event.get("data",{}).get("chunk",{})  
            _contents=base_message.content
            for i in range(len(_contents)):
                _content=_contents[i]
                if "type" in _content:
                    chunks.extend(self.__process_chunk(event, _content["type"], _content))
                    self.ref["last_type"] = _content["type"]
        elif self.ref["wrap_status"]=="started": # since this function called first, handle end of text or tool_use use for any other event
            allow_close=self.ref["last_type"] in ["text","tool_use"]
            if allow_close:
                chunks.extend(self.__process_chunk(event, event["event"], None))
                self.ref["last_type"]=None
                self.ref["wrap_status"]="ended"
                self.ref["last_id"]=None
        return chunks
        
    def __process_non_chunks(self,event):
        ac_events=[]
        if not isinstance(event.get("data",{}).get("chunk",{}),langchain_messages.BaseMessage):
            if event["event"]=="on_tool_start":
                run_id = event["run_id"]
                # Initialize buffer for this tool run
                self.tool_event_buffer[run_id] = []
                self.active_tool_runs[run_id] = True
                
                # Buffer the tool start event
                tool_start_event = ToolCallStartEvent(
                    type=EventType.TOOL_CALL_START,
                    tool_call_name=event["name"],
                    tool_call_id=run_id,
                    raw_event=event
                )
                self.tool_event_buffer[run_id].append(tool_start_event)
                
                tool_args_event = ToolCallArgsEvent(
                    type=EventType.TOOL_CALL_ARGS,
                    delta=json.dumps(event["data"]["input"], default=str),
                    tool_call_id=run_id,
                    raw_event=event
                )
                self.tool_event_buffer[run_id].append(tool_args_event)
                
            elif event["event"]=="on_tool_end":
                run_id = event["run_id"]
                
                # Buffer the tool end event
                tool_end_event = ToolCallEndEvent(
                    type=EventType.TOOL_CALL_END,
                    tool_call_id=run_id,
                    raw_event=event
                )
                
                # If this run_id exists in buffer, add end event and return all buffered events
                if run_id in self.tool_event_buffer:
                    self.tool_event_buffer[run_id].append(tool_end_event)
                    # Return all buffered events for this run_id
                    ac_events.extend(self.tool_event_buffer[run_id])
                    # Clean up buffer
                    del self.tool_event_buffer[run_id]
                    self.active_tool_runs.pop(run_id, None)
                else:
                    # Fallback: just add the end event if buffer doesn't exist
                    ac_events.append(tool_end_event)
                
            elif event.get("data",{}).get("chunk",{}).get("__interrupt__",False):
                _interrupt:Interrupt= event["data"]["chunk"]["__interrupt__"]
                ac_events.append(CustomEvent(
                    type=EventType.CUSTOM,
                    name="on_interrupt",
                    value={
                        "text": _interrupt[0].value,
                        "id": _interrupt[0].id,
                        "type": "on_interrupt"
                    },
                    raw_event=event
                ))
            else:
                print("Unhandled event:", event)
        return ac_events

    async def __handle_event(self,event):
        chunks=[]
        chunks.extend(self.__process_chunks(event))
        chunks.extend(self.__process_non_chunks(event))           
        if chunks:
            for chunk in chunks:
                yield chunk

    async def transform_events(self,event):  # !event: don't modify event it will have reference
        # event=copy.deepcopy(event) # !Caution, avoid modifying langgraph data, since it will have reference to all its internal variables
        _event=self.__get_filtered_data(event)
        self.filtered_event=_event
        if _event is None:
            yield None
        else:
            async for transformed_event in self.__handle_event(_event):
                yield transformed_event

    def __end_events(self):
        # Handle any remaining buffered tool events
        remaining_events = []
        for run_id, buffered_events in self.tool_event_buffer.items():
            print(f"Warning: Tool run {run_id} did not complete properly, flushing buffered events")
            remaining_events.extend(buffered_events)
        
        # Clear all buffers
        self.tool_event_buffer.clear()
        self.active_tool_runs.clear()
        
        if self.ref["wrap_status"]=="started":
            if self.ref["last_type"]=="text":
                end_event = TextMessageEndEvent(
                    type=EventType.TEXT_MESSAGE_END,
                    message_id=self.ref["last_id"] or ""
                )
                remaining_events.append(end_event)
            elif self.ref["last_type"]=="tool_use":
                end_event = CustomEvent(
                    type=EventType.CUSTOM,
                    name=self.ref["last_type"],
                    value={"text":"","type":EventType.TEXT_MESSAGE_END,"message_id":self.ref["last_id"] or ""}
                )
                remaining_events.append(end_event)
            else:
                print("Unhandled end event type:", self.ref["last_type"])
                end_event = RawEvent(
                    type=EventType.RAW,
                    event=f"Unhandled end event type: {self.ref['last_type']}"
                )
                remaining_events.append(end_event)
        else:
            if not remaining_events:
                print("No active wrap status to end:", self.ref["wrap_status"])
                remaining_events.append(RawEvent(
                    type=EventType.RAW,
                    event="No active wrap status to end"
                ))
        
        return remaining_events

    def end_events(self):
        data=self.__end_events()
        print("event_types", self.event_types)
        print("chunk_types", self.chunk_types)
        print("parsed_events", self.parsed_events)
        print("unparsed_events", self.unparsed_events)
        self.ref= {"wrap_status": None, "last_type": None}
        return data

ag_transformers = LangGraphToAgUi()
try:
    encoder = EventEncoder(accept="text/event-stream")
    final_events=[]
    filtered_event=[]
    print(len(all_events))
    with open("original_events.json", "w") as file:
        file.write(json.dumps(all_events, default=str,indent=2))
    for event in all_events:
        processed_events=ag_transformers.transform_events(event)
        filtered_event.append(ag_transformers.filtered_event)
        async for transformed_event in processed_events:
            if transformed_event:
                final_events.append(encoder.encode(transformed_event))
    
    # Handle end events - this might return multiple events now
    end_events = ag_transformers.end_events()
    if isinstance(end_events, list):
        for end_event in end_events:
            final_events.append(encoder.encode(end_event))
    else:
        final_events.append(encoder.encode(end_events))
    
    print("Final Events Count:", len(final_events))
    with open("filtered_data.json", "w") as file:
        file.write(json.dumps(filtered_event, default=str,indent=2))
    with open("final_events.json", "w") as file:
        file.write(json.dumps(final_events, default=str,indent=2))
except Exception as e:
    traceback.print_exc(limit=None)

Exception reporting mode: Verbose
5729
fbc4b0f1-e30c-4e43-b1f8-a880dcf150c9 tool_use_7e5bd722-eee2-41a5-9f7e-8afcc155ae3b
16abbaa7-265c-45fd-b1d0-f64645f1c6dd tool_use_fbc4b0f1-e30c-4e43-b1f8-a880dcf150c9
ecb3bd92-489f-463b-95bc-f22282926253 tool_use_d7cc9834-c9ca-41e7-bc61-bd7577902c6e
0e10cadc-4835-4480-9ff2-8f766f35a3f2 tool_use_9b1392ea-ca69-43e6-b95d-be8739623950
230c9588-36f4-41e9-8097-d68316880f03 tool_use_71063174-d3e0-46bf-9128-436d840b8579
5ae2aafb-b446-45fe-b15b-c6daea3b8071 text_952e3be5-cecf-446c-84ee-5669f3f5ae03
4bcd7e40-f7c7-4421-83c2-8cf6fccd891e tool_use_5ae2aafb-b446-45fe-b15b-c6daea3b8071
a2910e45-d772-492f-a6e5-1d912234cd8f tool_use_fd8058f7-7b46-481d-b67f-963aa65737db
a32a5419-0e79-4165-8d13-3104cab01c95 tool_use_d4b94030-d7ea-4b84-8493-5135731eb431
b67c8b89-6487-45cb-815a-35f5fd9b5c98 tool_use_e77ea700-2b0b-4120-b575-07fc11053a9b
301642e9-99f8-4522-a984-36f87f5e9af1 tool_use_d8748349-5e5c-42c4-9672-f988c4671ff5
b7c4cd51-6978-4e3a-bffd-016a257e7910 tool_use_18c249